In [6]:
# training_demo.ipynb

# 1. Imports
from pathlib import Path
import torch
from torch import nn
from torch.nn import Module
from torch.utils.data import Dataset
import torchvision.transforms as T
from diffusers.models import AutoencoderKL
from transfusion_pytorch import Transfusion, print_modality_sample
# Image and model size balancing
IMAGE_SIZE = 512  # Base image size
LATENT_FACTOR = 8  # SD VAE's downsampling factor
LATENT_SIZE = IMAGE_SIZE // LATENT_FACTOR  # = 96x96 latent space

In [7]:
# 2. Helper Classes
class Encoder(Module):
    def __init__(self, vae):
        super().__init__()
        self.vae = vae

    def forward(self, image):
        with torch.no_grad():
            latent = self.vae.encode(image * 2 - 1)
        return 0.18215 * latent.latent_dist.sample()

class Decoder(Module):
    def __init__(self, vae):
        super().__init__()
        self.vae = vae

    def forward(self, latents):
        latents = (1 / 0.18215) * latents
        with torch.no_grad():
            image = self.vae.decode(latents).sample
        return (image / 2 + 0.5).clamp(0, 1)

# 3. Model Setup
def setup_model():
    # Load VAE
    vae = AutoencoderKL.from_pretrained("CompVis/stable-diffusion-v1-4", subfolder="vae")
    
    # Initialize model
    model = Transfusion(
        num_text_tokens = 256,
        dim_latent = 4,
        channel_first_latent = True,
        modality_default_shape = (LATENT_SIZE, LATENT_SIZE),
        modality_encoder = Encoder(vae),
        modality_decoder = Decoder(vae),
        pre_post_transformer_enc_dec = (
            nn.Sequential(
                nn.Conv2d(4, 640, 3, 2, 1),
                nn.Conv2d(640, 1280, 3, 2, 1),
            ),
            nn.Sequential(
                nn.ConvTranspose2d(1280, 640, 3, 2, 1, output_padding=1),
                nn.ConvTranspose2d(640, 4, 3, 2, 1, output_padding=1),
            )
        ),
        add_pos_emb = True,
        modality_num_dim = 2,
        reconstruction_loss_weight = 0.05,
        transformer = dict(
            dim = 1280,
            depth = 24,
            dim_head = 80,
            heads = 16,
            dropout = 0.1,
            ff_expansion_factor = 2.0
        )
    ).cuda()

    return model

In [8]:

# 4. Load Checkpoint
def load_checkpoint(model, checkpoint_path):
    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    return model

In [9]:
# 5. Generate Samples
def generate_samples(model, num_samples=4):
    with torch.no_grad():
        sample = model.sample(batch_size=num_samples)
        print_modality_sample(sample)
        return sample

# 6. Demo Usage
if __name__ == "__main__":
    # Setup model
    model = setup_model()
    
    # Load checkpoint
    checkpoint_path = Path('./checkpoints/best_model.pt')
    if checkpoint_path.exists():
        model = load_checkpoint(model, checkpoint_path)
        print("Loaded checkpoint successfully!")
    
    # Generate samples
    samples = generate_samples(model)

/tmp/ipykernel_106718/3328559512.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path)


RuntimeError: Error(s) in loading state_dict for Transfusion:
	Missing key(s) in state_dict: "transformer.layers.12.0.weight", "transformer.layers.13.0.weight", "pos_emb_mlp.0.mlps.0.0.weight", "pos_emb_mlp.0.mlps.0.0.bias", "pos_emb_mlp.0.mlps.0.2.weight", "pos_emb_mlp.0.mlps.0.2.bias", "pos_emb_mlp.0.mlps.0.4.weight", "pos_emb_mlp.0.mlps.0.4.bias", "pos_emb_mlp.0.mlps.1.0.weight", "pos_emb_mlp.0.mlps.1.0.bias", "pos_emb_mlp.0.mlps.1.2.weight", "pos_emb_mlp.0.mlps.1.2.bias", "pos_emb_mlp.0.mlps.1.4.weight", "pos_emb_mlp.0.mlps.1.4.bias", "latent_to_model_projs.0.0.0.weight", "latent_to_model_projs.0.0.0.bias", "latent_to_model_projs.0.0.1.weight", "latent_to_model_projs.0.0.1.bias", "model_to_latent_projs.0.1.0.weight", "model_to_latent_projs.0.1.0.bias", "model_to_latent_projs.0.1.1.weight", "model_to_latent_projs.0.1.1.bias". 
	Unexpected key(s) in state_dict: "transformer.layers.24.0.weight", "transformer.layers.24.1.layernorm_gamma", "transformer.layers.24.1.layerscale", "transfor